# 2.3.3 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Best results for text_prompts: A2, B1, C1, D1
* Best results for img_prompts: B2, C2, D2, E3

### Setup

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
samantha = OpenAI(instructions)
parser = Parser()
runner = Runner()

In [2]:
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print("Seitenzahl:", len(pdf_prompts))

Seitenzahl: 66


### A) OCR komplettes PDF

In [3]:
instruction = 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
instruction_prompt = parser.text2prompts([instruction])

In [ ]:
# A1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Komplettes PDF auf einmal einlesen. Ziel: Konsistentes Layout.
pdf_text = samantha.request([*pdf_prompts, *instruction_prompt])
pprint(pdf_text)

In [ ]:
# A2) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten. Ziel: Fokus maximieren und Context-Window klein halten + Seitenzahlen beleiben erhalten.
prompt_chain = list(map(lambda prompt: samantha.lambdaRequest([prompt, *instruction_prompt]), pdf_prompts))
msgbp872apdf = await asyncio.gather(*prompt_chain)
pprint(msgbp872apdf)
%store msgbp872apdf

### B) Art der baulichen Nutzung

1. text_prompts
2. img_prompts

In [16]:
%store -r msgbaunvo_art msgbp872apdf
context_art = parser.text2prompts([msgbaunvo_art])

In [19]:
# B1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'

def run():
    return samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_art)
runner.consistency_check(run)

['Hier sind alle Informationen zum Thema "Art der baulichen Nutzung" aus dem bereitgestellten Text extrahiert:

### § 4 Art der baulichen Nutzung
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

**Zulässig sind:**
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

**Nicht zulässig sind:**
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

### D.5.2.1. Art der baulichen Nutzung

#### D.5.2.1.1. Allgemeines Wohngebiet
Nachdem das aktuelle Planungskonzept vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebietes vorsieht, wird anstatt der im rechtskräftigen BP Nr. 872 festgesetzten Fläche für den Gemeinbedarf mit der Zweckbe

[['Hier sind alle Informationen zum Thema "Art der baulichen Nutzung" aus dem bereitgestellten Text extrahiert:\n\n### § 4 Art der baulichen Nutzung\n(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.\n\n**Zulässig sind:**\n- Wohngebäude,\n- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,\n- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.\n\n**Nicht zulässig sind:**\n- Betriebe des Beherbergungsgewerbes,\n- sonstige nicht störende Gewerbebetriebe,\n- Anlagen für Verwaltungen,\n- Gartenbaubetriebe,\n- Tankstellen.\n\n### D.5.2.1. Art der baulichen Nutzung\n\n#### D.5.2.1.1. Allgemeines Wohngebiet\nNachdem das aktuelle Planungskonzept vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebietes vorsieht, wird anstatt der im rechtskräftigen BP Nr. 872 festgesetzten Fläche für den Gemeinb

In [18]:
# B2) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Art der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'

def run():
    return samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_art)
results = runner.consistency_check(run)
msg233_art = results[0]
%store msg233_art

Die Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan Nr. 872 A, „Zwischen Waldstraße und Döllgaststraße“, sind wie folgt:

### § 4 Art der baulichen Nutzung

1. **Allgemeines Wohngebiet (WA)**
   - Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
   - **Zulässig sind:**
     - Wohngebäude
     - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe, die der Versorgung des Gebietes dienen
     - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
   - **Nicht zulässig sind:**
     - Betriebe des Beherbergungsgewerbes
     - Sonstige nicht störende Gewerbebetriebe
     - Anlagen für Verwaltungen
     - Gartenbaubetriebe
     - Tankstellen

2. **Fremdwerbeanlagen**
   - Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### D.5.2.1.1. Allgemeines Wohngebiet

- Das aktuelle Planungskonzept sieht vorwiegend eine

In [22]:
# B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF

instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'

async def run():
    return await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
await runner.async_consistency_check(run)

['### Art der baulichen Nutzung: Informationen

#### § 4 Art der baulichen Nutzung
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

Zulässig sind:
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

Nicht zulässig sind:
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

#### D.5.2.1.1. Allgemeines Wohngebiet
Nach dem aktuellen Planungskonzept wird vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebiets vorgesehen. Die Festsetzungen zu den im Planbereich ausgewiesenen Wohnnutzungen orientieren sich dabei am Nutzungskatalog der allgemeinen Wohngebiete gemäß 

[['### Art der baulichen Nutzung: Informationen\n\n#### § 4 Art der baulichen Nutzung\n(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.\n\nZulässig sind:\n- Wohngebäude,\n- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,\n- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.\n\nNicht zulässig sind:\n- Betriebe des Beherbergungsgewerbes,\n- sonstige nicht störende Gewerbebetriebe,\n- Anlagen für Verwaltungen,\n- Gartenbaubetriebe,\n- Tankstellen.\n\n(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.\n\n#### D.5.2.1.1. Allgemeines Wohngebiet\nNach dem aktuellen Planungskonzept wird vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebiets vorgesehen. Die Festsetzungen zu den im Planbereich ausgewiesenen Wohnnutzungen orientieren sich dabei am Nutzungskatalog der allgemein

### C) Maß der baulichen Nutzung

1. text_prompts
2. img_prompts

In [15]:
%store -r msgbaunvo_maß msgbp872apdf
context_maß = parser.text2prompts([msgbaunvo_maß])

In [21]:
# C1) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'
def run():
    return samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_maß)
runner.consistency_check(run)

['**Maß der baulichen Nutzung**

Das Maß der baulichen Nutzung wird im allgemeinen Wohngebiet durch die Festlegung der höchstzulässigen Grundflächenzahl (GRZ) und der höchstzulässigen Geschossfläche (GF), durch die Zahl der Vollgeschosse (Höchstmaß bzw. zwingend) und durch die Höhe der baulichen Anlagen (OK, Höchstmaß bzw. zwingend) innerhalb der einzelnen überbaubaren Grundstücksflächen (Baufelder) ausreichend bestimmt.

**§ 5 Maß der baulichen Nutzung**

1. Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
2. In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
    - WA1: GRZ maximal 0,65
    - WA2: GRZ maximal 0,80
    - WA3: GRZ maximal 0,75
    - WA4: GRZ maximal 0,75

**§ 6 Höhenlage, Bezugspunkte**

Mit der Zulässigkeit einer Überschreitung der festgesetzten Gebäudeoberkanten (OK) durch 

[['**Maß der baulichen Nutzung**\n\nDas Maß der baulichen Nutzung wird im allgemeinen Wohngebiet durch die Festlegung der höchstzulässigen Grundflächenzahl (GRZ) und der höchstzulässigen Geschossfläche (GF), durch die Zahl der Vollgeschosse (Höchstmaß bzw. zwingend) und durch die Höhe der baulichen Anlagen (OK, Höchstmaß bzw. zwingend) innerhalb der einzelnen überbaubaren Grundstücksflächen (Baufelder) ausreichend bestimmt.\n\n**§ 5 Maß der baulichen Nutzung**\n\n1. Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.\n2. In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:\n    - WA1: GRZ maximal 0,65\n    - WA2: GRZ maximal 0,80\n    - WA3: GRZ maximal 0,75\n    - WA4: GRZ maximal 0,75\n\n**§ 6 Höhenlage, Bezugspunkte**\n\nMit der Zulässigkeit einer Überschreitung der festgesetzten Gebäudeoberka

In [14]:
# C2) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Maß der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'

def run():
    return samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_maß)
results = runner.consistency_check(run)
msg233_maß = results[0]
%store msg233_maß

Hier sind alle Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan:

### § 5 Maß der baulichen Nutzung

1. **Festsetzung des Maßes der baulichen Nutzung:**
   - Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

2. **Zulässige Grundflächenzahl (GRZ) in allgemeinen Wohngebieten:**
   - WA1: GRZ maximal 0,65
   - WA2: GRZ maximal 0,80
   - WA3: GRZ maximal 0,75
   - WA4: GRZ maximal 0,75

### D.5.2.1.2 Maß der baulichen Nutzung

1. **Allgemeine Festsetzungen:**
   - Das Maß der baulichen Nutzung wird im allgemeinen Wohngebiet durch die Festlegung der höchstzulässigen Grundflächenzahl (GRZ) und der höchstzulässigen Geschossfläche (GF), durch die Zahl der Vollgeschosse (Höchstmaß bzw. zwingend) und durch die Höhe der baulichen Anlagen (OK, Höchstmaß bzw. zwingend) innerhalb der einzelnen überbaubaren Grundstücksflächen (Baufelder) ausreichend bestimmt.

2. **GRZ-Festsetzungen:**
   - WA1, 3 und 4: höchstzulässige GR

### D) Bauweise, überbaubare Grundstücksflächen

1. text_prompts
2. img_prompts

In [6]:
%store -r msgbaunvo_bg msgbp872apdf
context_bg = parser.text2prompts([msgbaunvo_bg])

In [9]:
# D1) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'

def run():
    return samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_bg)
runner.consistency_check(run)

['### Bauweise, überbaubare Grundstücksflächen

#### § 22 Bauweise
1. Im Bebauungsplan kann die Bauweise als offene oder geschlossene Bauweise festgesetzt werden.
2. In der offenen Bauweise werden die Gebäude mit seitlichem Grenzabstand als Einzelhäuser, Doppelhäuser oder Hausgruppen errichtet.
3. In der geschlossenen Bauweise werden die Gebäude ohne seitlichen Grenzabstand errichtet.

#### § 23 Überbaubare Grundstücksfläche
1. Die überbaubaren Grundstücksflächen können durch die Festsetzung von Baugrenzen, Baulinien oder Bebauungstiefen bestimmt werden.
2. Ist eine Baulinie festgesetzt, so muss auf dieser Linie gebaut werden.
3. Ist eine Baugrenze festgesetzt, so dürfen Gebäude und Gebäudeteile diese nicht überschreiten.

#### D.5.2.3.2 Bauweise, überbaubare Grundstücksflächen
1. Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.
2. Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festgesetzt.
3. Vorbau

[['### Bauweise, überbaubare Grundstücksflächen\n\n#### § 22 Bauweise\n1. Im Bebauungsplan kann die Bauweise als offene oder geschlossene Bauweise festgesetzt werden.\n2. In der offenen Bauweise werden die Gebäude mit seitlichem Grenzabstand als Einzelhäuser, Doppelhäuser oder Hausgruppen errichtet.\n3. In der geschlossenen Bauweise werden die Gebäude ohne seitlichen Grenzabstand errichtet.\n\n#### § 23 Überbaubare Grundstücksfläche\n1. Die überbaubaren Grundstücksflächen können durch die Festsetzung von Baugrenzen, Baulinien oder Bebauungstiefen bestimmt werden.\n2. Ist eine Baulinie festgesetzt, so muss auf dieser Linie gebaut werden.\n3. Ist eine Baugrenze festgesetzt, so dürfen Gebäude und Gebäudeteile diese nicht überschreiten.\n\n#### D.5.2.3.2 Bauweise, überbaubare Grundstücksflächen\n1. Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.\n2. Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festges

In [8]:
# D2)Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'

def run():
    return samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_bg)
results = runner.consistency_check(run)
msg233_bg = results[0]
%store msg233_bg

### § 7 Bauweise, Überbaubare Grundstücksflächen, Abstandsflächen

(1) Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.

(2) Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festgesetzt.

(3) Vorbauten dürfen die Baugrenzen bis zu 1,5 m auf einer Breite von maximal 5,0 m je Eingang überschreiten. Balkone dürfen mit Ausnahme des jeweils obersten Geschosses die Baugrenzen bis zu 0,90 m auf einer Breite von maximal 7,50 m je Balkon überschreiten. Zwischen den einzelnen Balkonen muss ein Abstand von mindestens 3,25 m eingehalten werden.

(4) Terrassen an Gebäuden dürfen sich bis zu einer Tiefe von 2,5 m auch auf Flächen außerhalb der überbaubaren Grundstücksflächen erstrecken. Sie dürfen außerhalb der überbaubaren Grundstücksflächen nicht durch Wintergärten oder ähnliches überbaut werden.

(5) Im Kronenbereich der in der Planzeichnung (Teil A) dargestellten und festgesetzten Bestandsgehölze sind die in Abs

### E) Alle Themen in einem Prompt

1. text_prompts (TODO)
2. img_prompts

In [11]:
%store -r msgbaunvo msgbp872apdf
context_all = parser.text2prompts([msgbaunvo])

In [5]:
# E1) Textinformationen: Alle Themen – WITH CONTEXT – IMG_PROMPTS

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_all], img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung

**§ 4 Art der baulichen Nutzung**
1. Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
   - **Zulässig sind:**
     - Wohngebäude
     - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe, die der Versorgung des Gebietes dienen
     - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
   - **Nicht zulässig sind:**
     - Betriebe des Beherbergungsgewerbes
     - Sonstige nicht störende Gewerbebetriebe
     - Anlagen für Verwaltungen
     - Gartenbaubetriebe
     - Tankstellen

2. Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung

**§ 5 Maß der baulichen Nutzung**
1. Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
2. In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen de

In [6]:
# E2) Textinformationen: Alle Themen – WITHOUT CONTEXT – IMG_PROMPTS
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
messages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung:
- Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
- Zulässig sind:
  - Wohngebäude
  - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe, die der Versorgung des Gebietes dienen
  - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
- Nicht zulässig sind:
  - Betriebe des Beherbergungsgewerbes
  - sonstige nicht störende Gewerbebetriebe
  - Anlagen für Verwaltungen
  - Gartenbaubetriebe
  - Tankstellen
  - Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung:
- Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
- In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
  - WA1: GRZ 

In [23]:
# TODO WITH CONTEXT!!!

# E3) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis) – WITHOUT CONTEXT
# Wenn Dokument zu groß wird: z.B. chunk_size=10 pages, overlap=2 pages.
# chunk_size = 10, overlap = 2
# n = math.ceil(len(pdf_prompts)/chunk_size)
# chunks = list(map(lambda i: pdf_prompts[slice(max(i*chunk_size-overlap,0),i*chunk_size+chunk_size)], range(n)))
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zu folgenden Themen beinhalten: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'

async def run():
    return await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
await runner.async_consistency_check(run)
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all

# instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche.'
# msg233_all = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_all)
# pprint(msg233_all)
# %store msg233_all

```json
{
  "Seiten": [6, 7, 8]
}
```
#############################################
```json
{
  "Seiten": [6, 7, 8, 28, 29, 30, 31, 32, 33]
}
```
#############################################
```json
{
  "Seiten": [6, 7, 8, 28, 29, 30, 31, 32, 33]
}
```
#############################################


In [13]:
# E4) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis) – WITH CONTEXT
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche.'

def run():
    return  samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_all)
results = runner.consistency_check(run)
msg233_all = results[0]
%store msg233_all

### Art der baulichen Nutzung

#### § 4 Art der baulichen Nutzung
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

**Zulässig sind:**
- Wohngebäude
- Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe, die der Versorgung des Gebietes dienen
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke

**Nicht zulässig sind:**
- Betriebe des Beherbergungsgewerbes
- Sonstige nicht störende Gewerbebetriebe
- Anlagen für Verwaltungen
- Gartenbaubetriebe
- Tankstellen

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung

#### § 5 Maß der baulichen Nutzung
(1) Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

(2) In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anla

In [8]:
# E5) Mehrere Threads – WITHOUT CONTEXT
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
async def run(pdf_prompt):
    return samantha.request([
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Steht keine relevante Information zur Verfügung gebe "-" aus.'
        },
        pdf_prompt
    ])

prompt_chain = list(map(lambda pdf_prompt: run(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)
pprint(results)

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
#############################################
Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Inhaltsverzeichnis des Bebauungsplans:

- **Art der baulichen Nutzung**: -
- **Maß der baulichen Nutzung**: -
- **Bauweise**: -
- **Überbaubare Grundstücksfläche**: -

Das Inhaltsverzeichnis enthält keine spezifischen Informationen zu diesen Themen. Um detaillierte Informationen zu erhalten, müsste der vollständige Text des Bebauungsplans eingesehen werden.
#############################################
Die angeforderte Seite enthält keine Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche". Es handelt sich um eine Seite mit Abkürzungen und deren Erläuterungen.

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
######